In [253]:
import pandas as pd
import json
import os



In [254]:
pwd

'C:\\Users\\Jeff\\Desktop\\Data Bootcamp Files\\Homework 4 Pandas'

In [255]:
data = pd.read_json("purchase_data.json",orient="records")

In [256]:
data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [257]:
total_players = data["SN"].nunique()
total_players_df = pd.DataFrame({"Total Players": [total_players]})

total_players_df.head()

,Total Players
0,573


In [406]:
num_of_items = data["Item ID"].nunique()
average_price = data["Price"].mean()
number_of_purch = len(data)
total_revenue = data["Price"].sum()
purch_analysis_df = pd.DataFrame({"Number of Unique Items": [num_of_items], "Average Price": [average_price], "Number of Purchases": [number_of_purch], "Total Revenue": [total_revenue]})

purch_analysis_df["Average Price"] = purch_analysis_df["Average Price"].map("${:.2f}".format)
purch_analysis_df["Total Revenue"] = purch_analysis_df["Total Revenue"].map("${:.2f}".format)

purch_analysis_df.head()

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.93,780,183,$2286.33


In [259]:
male_total = len(data[data["Gender"] == 'Male'])
female_total = len(data[data["Gender"] == 'Female'])
other_total = len(data[data["Gender"] == 'Other / Non-Disclosed'])
all_total = male_total + female_total + other_total

gender_demo_df = pd.DataFrame({"":["Male", "Female", "Other / Non-Disclosed"], "Percentage of Players": [(male_total/all_total)*100, (female_total/all_total)*100, (other_total/all_total)*100], "Total Count": [male_total, female_total, other_total]})
gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:.2f}%".format)
gender_demo_df

,,Percentage of Players,Total Count
0,Male,81.15%,633
1,Female,17.44%,136
2,Other / Non-Disclosed,1.41%,11


In [404]:
gender_data = pd.DataFrame(data.groupby('Gender')['Item ID'].count())
gender_data['Average Purchase Price'] = data.groupby('Gender')['Price'].mean()
gender_data['Total Purchase Value'] = data.groupby('Gender')['Price'].sum()

norm_data = pd.DataFrame(data.groupby('Gender')['SN'].nunique())
gender_data['Normalized Totals'] =  data.groupby('Gender')['Price'].sum()/ norm_data['SN']
gender_data.rename(columns={'Item ID':'Purchase Count'}, inplace=True)

gender_data["Average Purchase Price"] = gender_data["Average Purchase Price"].map("${:.2f}".format)
gender_data["Total Purchase Value"] = gender_data["Total Purchase Value"].map("${:.2f}".format)
gender_data["Normalized Totals"] = gender_data["Normalized Totals"].map("${:.2f}".format)

gender_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,$1867.68,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [274]:
bins = [0, 10, 15, 20, 25, 30, 35, 39, 40]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
data["Age Groups"] = pd.cut(data["Age"], bins, labels=group_names)
ages_count = data.groupby('Age Groups')['Gender'].count()
ages_analysis = pd.DataFrame(ages_count)
ages_analysis["Percentage of Players"] = ages_analysis["Gender"]/ all_total *100
ages_analysis["Percentage of Players"] = ages_analysis["Percentage of Players"].map("{0:,.2f}%".format)
ages_analysis.rename(columns={'Gender':'Total Count'}, inplace=True)

ages_analysis
 



,Total Count,Percentage of Players
Age Groups,,
<10,32,4.10%
10-14,78,10.00%
15-19,184,23.59%
20-24,305,39.10%
25-29,76,9.74%
30-34,58,7.44%
35-39,30,3.85%
40+,14,1.79%


In [409]:
purch_item_count = pd.DataFrame(data.groupby('Age Groups')['Item ID'].count())
purch_item_count.rename(columns={"Item ID":'Purchase Count'}, inplace=True)
purch_item_count['Average Purchase Price'] = data.groupby('Age Groups')['Price'].sum()/ purch_item_count['Purchase Count']
purch_item_count['Total Purchase Price'] = data.groupby('Age Groups')['Price'].sum()

norm_count = pd.DataFrame(data.groupby('Age Groups')['SN'].nunique())
purch_item_count['Normalized Totals'] =  data.groupby('Age Groups')['Price'].sum()/ norm_count['SN']

purch_item_count["Normalized Totals"] = purch_item_count["Normalized Totals"].map("${:.2f}".format)
purch_item_count["Average Purchase Price"] = purch_item_count["Average Purchase Price"].map("${:.2f}".format)
purch_item_count["Total Purchase Price"] = purch_item_count["Total Purchase Price"].map("${:.2f}".format)

purch_item_count.head()

,Purchase Count,Average Purchase Price,Total Purchase Price,Normalized Totals
Age Groups,,,,
<10,32,$3.02,$96.62,$4.39
10-14,78,$2.87,$224.15,$4.15
15-19,184,$2.87,$528.74,$3.80
20-24,305,$2.96,$902.61,$3.86
25-29,76,$2.89,$219.82,$4.23


In [410]:
spenders = pd.DataFrame(data.groupby('SN')['Price'].sum())
spenders = spenders.sort_values('Price', ascending=False)
spenders['Purchase Count'] = data.groupby('SN')['Item ID'].count()
spenders['Average Purchase Price'] = spenders['Price']/ spenders['Purchase Count']

spenders["Average Purchase Price"] = spenders["Average Purchase Price"].map("${:.2f}".format)
spenders["Price"] = spenders["Price"].map("${:.2f}".format)

spenders.head()

,Price,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,$17.06,5,$3.41
Saedue76,$13.56,4,$3.39
Mindimnya67,$12.74,4,$3.18
Haellysu29,$12.73,3,$4.24
Eoda93,$11.58,3,$3.86


In [418]:
top_items = pd.DataFrame(data.groupby('Item ID')['SN'].count())
top_items = top_items.sort_values('SN', ascending=False)
top_items["Item Name"] = data['Item Name']
top_items["Item Price"] = data['Price']
top_items["Total Purchase Value"] = top_items['SN']* top_items['Item Price']
top_items.rename(columns={"SN":'Purchase Count'}, inplace=True)

top_items["Item Price"] = top_items["Item Price"].map("${:.2f}".format)
top_items["Total Purchase Value"] = top_items["Total Purchase Value"].map("${:.2f}".format)
top_items = top_items[['Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']]

top_items.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
39,Stormfury Mace,11,$1.27,$13.97
84,"Thorn, Satchel of Dark Souls",11,$4.51,$49.61
31,"Shadow Strike, Glory of Ending Hope",9,$1.93,$17.37
175,Retribution Axe,9,$4.14,$37.26
13,"Piety, Guardian of Riddles",9,$3.68,$33.12


In [425]:
top_value = pd.DataFrame(data.groupby('Item ID')['SN'].count())

top_value["Item Name"] = data['Item Name']
top_value["Item Price"] = data['Price']

top_value.rename(columns={"SN":'Purchase Count'}, inplace=True)
top_value["Total Purchase Value"] = top_value['Purchase Count'] * top_value['Item Price']

top_value = top_value.sort_values('Total Purchase Value', ascending=False)
top_value["Item Price"] = top_value["Item Price"].map("${:.2f}".format)
top_value["Total Purchase Value"] = top_value["Total Purchase Value"].map("${:.2f}".format)
top_value = top_value[['Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']]
top_value.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
84,"Thorn, Satchel of Dark Souls",11,$4.51,$49.61
65,Undead Crusader,8,$4.67,$37.36
175,Retribution Axe,9,$4.14,$37.26
92,Reaper's Toll,8,$4.56,$36.48
107,Spectral Diamond Doomblade,8,$4.25,$34.00
